In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('/content/drive/My Drive/APT/newest/TAR-project-master')

In [3]:
pip install num2words

In [4]:
pip install langdetect

In [5]:
pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.2.6.1)


In [6]:
import pandas as pd
import numpy as np
import nltk

nltk.download('stopwords')
nltk.download('punkt')

from dataset.preprocessing.preprocessing import word_stem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
abstracts_path = '/content/drive/My Drive/APT/temp_data/english_covid_abstracts_data.csv'
abstracts = pd.read_csv(abstracts_path)
abstracts.head()

,paper_id,section,text,preprocessed_text
0,86a998617c077f4fe2ab26214995a3548fbc0fc5,Abstract,The recent emergence of the Middle East respir...,the recent emerg middl east respiratori syndro...
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,Abstract,Thousands of people in the United States have ...,thousand peopl unit state requir test sars-cov...
2,5e0c586f047ff909c8ed3fe171c8975a90608d08,Abstract,Background: Porcine epidemic diarrhea virus (P...,background porcin epidem diarrhea viru pedv em...
3,1579fbff7af9b156c6f49fee0526e48f852ea460,Abstract,"Currently, live-attenuated IBV vaccines are us...",current live-attenu ibv vaccin use control dis...
4,f6b29be971089bfe0916c64ab9fbddcec38a7436,Abstract,Aims: To determine analytical capabilities of ...,aim determin analyt capabl commonli use faecal...


## Queries

In [0]:
queries = [
           "incubation period of COVID-19",
           "Does smoking increase risks when having covid19?",
           "mortality rate of covid19?",
           "pregnancy complications covid19?"
]

def run_queries(query_engine, queries, k=5):
  results = query_engine.run_query(queries[0], k)
  if len(queries) > 1:
    for q in queries[1:]:
      results = pd.concat([results, query_engine.run_query(q, k)])
  return results

## TF-IDF

In [0]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/tf_idf.dat')

In [37]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'tf_idf')

Writing object to /content/drive/My Drive/APT/trained_models/tf_idf.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## BM25

In [0]:
from nltk.corpus import stopwords
from query_model.query_engines import BOWQueryEngine
from query_model.transformers.bm25 import BM25Transformer

In [0]:
stop_words = set(stopwords.words('english'))
cv = CountVectorizer(stop_words=stop_words)
transformer = BM25Transformer()
query_engine = BOWQueryEngine(cv, transformer)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
# query_engine = BOWQueryEngine.load('/content/drive/My Drive/APT/trained_models/bm25.dat')

In [23]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'bm25')

Writing object to /content/drive/My Drive/APT/trained_models/bm25.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## Word2Vec

In [0]:
from query_model.query_engines import W2VQueryEngine

In [0]:
# TODO train...
params = {
    'min_count': 5,
    'size': 300, 
    'workers': 3, 
    'window':3,
    'sg': 1
    }
query_engine = W2VQueryEngine(params)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
#query_engine = W2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/w2v.dat')

In [17]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'w2v')

Writing object to /content/drive/My Drive/APT/trained_models/w2v.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

## Doc2Vec

In [0]:
from query_model.query_engines import D2VQueryEngine

In [0]:
params = { 
    'vector_size': 300,
    'min_count': 5, 
    'negative': 5, 
    'hs': 0, 
    'sample': 1e-5,
    'epochs': 400,
    'window': 15
    }
query_engine = D2VQueryEngine(params)
query_engine.fit(abstracts)

In [0]:
# If you want to load already trained model
#query_engine = D2VQueryEngine.load('/content/drive/My Drive/APT/trained_models/d2v.dat')

In [43]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'd2v')

Writing object to /content/drive/My Drive/APT/trained_models/d2v.dat


In [24]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})

100%|██████████| 1/1 [00:00<00:00, 204.89it/s]


## BERT

In [0]:
from sentence_transformers import SentenceTransformer
from query_model.query_engines import BERTQueryEngine

In [46]:
import torch
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [47]:
query_engine = BERTQueryEngine()
query_engine.fit(abstracts)

100%|██████████| 405M/405M [00:45<00:00, 8.91MB/s]


In [0]:
# If you want to load already trained model
#query_engine = BERTQueryEngine.load('/content/drive/My Drive/APT/trained_models/BERT.dat')

In [0]:
query_engine.save('/content/drive/My Drive/APT/trained_models/', 'BERT')

Writing object to /content/drive/My Drive/APT/trained_models/BERT.dat


In [0]:
res = run_queries(query_engine, queries)
res.reset_index(drop=True)[['query', 'text', 'sim']].style.set_properties(**{'font-size': '9pt'})